# piont cloud separator

In [23]:
import numpy as np
from pyntcloud import PyntCloud

In [65]:
def find_ranges(point_cloud):

    x_max = x_min = point_cloud[0][0]
    y_max = y_min = point_cloud[0][1]
    z_max = z_min = point_cloud[0][2]
    
    for ind, coor in enumerate(point_cloud):
        if coor[0] > x_max:
            x_max = coor[0]
        if coor[0] < x_min:
            x_min = coor[0]
        if coor[1] > y_max:
            y_max = coor[1]
        if coor[1] < y_min:
            y_min = coor[1]
        if coor[2] > z_max:
            z_max = coor[2]
        if coor[2] < z_min:
            z_min = coor[2]
            
    return ((x_min, x_max), (y_min, y_max), (z_min, z_max))


def find_cube_length(t_range, overall_size):
    length = (t_range[1] - t_range[0])/overall_size
    return length

def get_all_length(point_cloud, overall_size, xyz_ranges):
    
    x_length = find_cube_length(xyz_ranges[0], overall_size[0])
    y_length = find_cube_length(xyz_ranges[1], overall_size[1])
    z_length = find_cube_length(xyz_ranges[2], overall_size[2])
    
    return (x_length, y_length, z_length)
    
def find_target_area_range(point_cloud, target_area, xyz_length, xyz_ranges):
    
    target_x_range_max = xyz_ranges[0][1] - target_area[0][0]*xyz_length[0]
    target_x_range_min = xyz_ranges[0][1] - target_area[0][1]*xyz_length[0]
    target_y_range_max = xyz_ranges[1][1] - target_area[1][0]*xyz_length[1]
    target_y_range_min = xyz_ranges[1][1] - target_area[1][1]*xyz_length[1]
    target_z_range_max = xyz_ranges[2][1] - target_area[2][0]*xyz_length[2]
    target_z_range_min = xyz_ranges[2][1] - target_area[2][1]*xyz_length[2]
    
    return (target_x_range_min, target_x_range_max), (target_y_range_min, target_y_range_max), (target_z_range_min, target_z_range_max)


def segment_points(point_cloud, target_ranges):
    """
    Recives orig point cloud data and a tuple of boundary tuples like:
        ((x_min, x_max), (y_min, y_max), (z_min, z_max))
    """
    
    target_x_range, target_y_range, target_z_range = target_ranges[0], target_ranges[1], target_ranges[2]
    
    points = []
    for ind, coor in enumerate(point_cloud):
        if coor[0] >= target_x_range[0] and coor[0] <= target_x_range[1]:
            if coor[1] >= target_y_range[0] and coor[1] <= target_y_range[1]:
                if coor[2] >= target_z_range[0] and coor[2] <= target_z_range[1]:
                    points.append(coor)
                    
    return np.asarray(points)

In [114]:
def traverse_point(point_cloud, cube_size=(3,3,3), overall_size=[32,32,32], stride=(1,1,1), filter_percentage=None):
    """
    It will return the target segment of point cloud.
    """
    threshold = 0
    
    if filter_percentage is not None:
        threshold = filter_percentage*len(point_cloud)
    
    xyz_range = find_ranges(point_cloud)
    xyz_length = get_all_length(point_cloud, overall_size, xyz_range)

    segmentations = []
    areas = []
    
    x_num = int((overall_size[0] - cube_size[0]) / stride[0] + 1)
    y_num = int((overall_size[1] - cube_size[1]) / stride[1] + 1)
    z_num = int((overall_size[2] - cube_size[2]) / stride[2] + 1)
    
    for _x in range(x_num):
        for _y in range(y_num):
            for _z in range(z_num):
                target_area = ((_x*stride[0], _x*stride[0] + cube_size[0]), (_y*stride[1], _y*stride[1] + cube_size[1]), (_z*stride[2], _z*stride[2] + cube_size[2]))
    
                target_x_range, target_y_range, target_z_range = find_target_area_range(point_cloud, target_area, xyz_length, xyz_range)
                
                segmented_point_cloud = segment_points(point_cloud, (target_x_range, target_y_range, target_z_range))
                
                if len(segmented_point_cloud) > threshold:
                    
                    segmentations.append(segmented_point_cloud)
                    
                    areas.append(target_area)
                    
                    
    if overall_size[0]%cube_size[0] is not 0 or overall_size[1]%cube_size[1] is not 0 or overall_size[2]%cube_size[2] is not 0:
        
        target_area = ((overall_size[0] - cube_size[0], overall_size[0]), (overall_size[1] - cube_size[1], overall_size[1]), (overall_size[2] - cube_size[2], overall_size[2]))
        
        target_x_range, target_y_range, target_z_range = find_target_area_range(point_cloud, target_area, xyz_length, xyz_range)

        segmented_point_cloud = segment_points(point_cloud, (target_x_range, target_y_range, target_z_range))
            
        if len(segmented_point_cloud) > threshold:
            
            segmentations.append(segmented_point_cloud)   
            
            areas.append(target_area)
    
    return {'data': segmentations, 'area': areas}

In [303]:
my_point_cloud = PyntCloud.from_file(os.path.join(os.getcwd(), 'ttt.pts'), sep=" ", header=0, names=["x","y","z"])

In [304]:
segs = traverse_point(my_point_cloud.xyz, cube_size=(6,6,6), overall_size=[13,13,13], stride=(1,1,1))

In [316]:
print(len(my_point_cloud.xyz), len(segs['data'][0]))
print(len(segs['area']))
vox_segs = []
whole_vox = voxelize3D(my_point_cloud_b.xyz, dim=[32,32,32])
whole_vox = whole_vox.reshape(whole_vox.shape + (1,))

for idx, value in enumerate(segs['data']):
    vox = voxelize3D(value, dim=[32,32,32])
    vox_chan = np.array(vox).reshape(vox.shape + (1,))
    vox_segs.append(vox_chan)

5167 1713
322


In [148]:
def voxelize3D(pts, dim=[1,1,1]):
    """
    pts: receives .pts cloud point data. 2D array, arbitary sized X,Y,Z pairs. (We will only take x,y,z into account for now)
    dim: dimensioin of output voxelized data
    
    This function will locate the grid cube and calculate the density of each cube.
    The output will be normalized values.
    """
    assert(pts.shape[1]>=3), "pts file should contain at least x,y,z coordinate"
    assert(len(dim)==3), "Please provide 3-d grid size like [32,32,32]"
    
    if len(pts) > 1:
        # move all the axis to positive area.
        minimum_val = [pts[0][0], pts[0][1], pts[0][2]]

        # find the smallest 
        for pair in pts:
            if pair[0] < minimum_val[0]:
                minimum_val[0] = pair[0]
            if pair[1] < minimum_val[1]:
                minimum_val[1] = pair[1]
            if pair[2] < minimum_val[2]:
                minimum_val[2] = pair[2]

        # move it to first quadrant 
        rectified_pts = np.empty(pts.shape)
        for index, pair in enumerate(pts):
            point = np.zeros(3)
            point[0] = pair[0] - minimum_val[0]
            point[1] = pair[1] - minimum_val[1]
            point[2] = pair[2] - minimum_val[2]
            rectified_pts[index] = point

        # biggest value in each axis 
        maximum_val = pts[0][0]

        for pair in rectified_pts:
            for val in pair:
                if val > maximum_val:
                    maximum_val = val

        # normalize all the axises to (0,1)
        normalized_pts = rectified_pts/maximum_val
    
    else:
        # in case there is just one point
        normalized_pts = pts
    
    x_grid_length = 1/dim[0]
    y_grid_length = 1/dim[1]
    z_grid_length = 1/dim[2]
    
    output = np.zeros((dim[0],dim[1],dim[2]))
    
    epsilon = 0.000000000001 # we will have at least a 1.0 value which will exceed the index of grid
    # we can use a relativly small value to escape that to fit our data
    
    max_volume_size = 0
    
    for pair in normalized_pts:
        x_loc = int(pair[0]/(x_grid_length + epsilon))
        y_loc = int(pair[1]/(y_grid_length + epsilon))
        z_loc = int(pair[2]/(z_grid_length + epsilon))
        if output[x_loc, y_loc, z_loc] is None:
            output[x_loc, y_loc, z_loc] = 1
        else:
            output[x_loc, y_loc, z_loc] += 1
        
        if output[x_loc, y_loc, z_loc] > max_volume_size:
            max_volume_size = output[x_loc, y_loc, z_loc]
    
    output = output/max_volume_size    
            
    return output

In [283]:
def count_v(array):
    li = {}
    for _, val in enumerate(array):
        _max = np.argmax(val)
        if _max not in li:
            li.update({_max:1})
        else:
            li.update({_max: li[_max]+1})
    return li

In [263]:
def get_previous_vars(model_path):
    
    new_graph = tf.Graph()
    config = tf.ConfigProto(allow_soft_placement=True) # allow passing gpu-trained model to a cpu machine
    
    with tf.Session(graph=new_graph, config=config) as sess:
        saver = tf.train.import_meta_graph(model_fqn + ".meta")
        saver.restore(sess, model_fqn)
        # Restore model
        previous_vars = {var.op.name for var in tf.global_variables()}

    return previous_vars

In [191]:
import tensorflow as tf
def cnn3d_model(x_train_data, label_size, stop_layer=0, keep_rate=0.7, seed=None):
    
    if seed is not None:
        tf.set_random_seed(seed)
        
    stop_config = np.ones(14, dtype=bool)
    for _ in range(stop_layer):
        stop_config[_] = False
    
    with tf.name_scope("layer_a"):
        # conv => 32*32*32
        conv1 = tf.layers.conv3d(inputs=x_train_data, filters=16, kernel_size=[3,3,3], padding='same',trainable=bool(stop_config[1]), activation=tf.nn.relu, name="conv1", reuse=tf.AUTO_REUSE)
        # conv => 32*32*32
        conv2 = tf.layers.conv3d(inputs=conv1, filters=32, kernel_size=[3,3,3], padding='same',trainable=bool(stop_config[2]), activation=tf.nn.relu, name="conv2", reuse=tf.AUTO_REUSE)
        # pool => 16*16*16
        pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[2, 2, 2], strides=2, name="pool3")
        
    with tf.name_scope("layer_b"):
        # conv => 16*16*16
        conv4 = tf.layers.conv3d(inputs=pool3, filters=64, kernel_size=[3,3,3], padding='same',trainable=bool(stop_config[4]), activation=tf.nn.relu, name="conv4", reuse=tf.AUTO_REUSE)
        # conv => 16*16*16
        conv5 = tf.layers.conv3d(inputs=conv4, filters=128, kernel_size=[3,3,3], padding='same',trainable=bool(stop_config[5]), activation=tf.nn.relu, name="conv5", reuse=tf.AUTO_REUSE)
        # pool => 8*8*8
        pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[2, 2, 2], strides=2, name="pool6")
        
    with tf.name_scope("layer_c"):
        # conv => 8*8*8
        conv7 = tf.layers.conv3d(inputs=pool6, filters=256, kernel_size=[3,3,3], padding='same',trainable=bool(stop_config[7]), activation=tf.nn.relu, name="conv7", reuse=tf.AUTO_REUSE)
        # conv => 8*8*8
        conv8 = tf.layers.conv3d(inputs=conv7, filters=512, kernel_size=[3,3,3], padding='same',trainable=bool(stop_config[8]), activation=tf.nn.relu, name="conv8", reuse=tf.AUTO_REUSE)
        # pool => 4*4*4
        pool9 = tf.layers.max_pooling3d(inputs=conv8, pool_size=[2, 2, 2], strides=2, name="pool9")
        
    with tf.name_scope("batch_norm"):
        cnn3d_bn = tf.layers.batch_normalization(inputs=pool9, trainable=bool(stop_config[10]), name="bn", reuse=tf.AUTO_REUSE)
        
    with tf.name_scope("fully_con"):
        flattening = tf.reshape(cnn3d_bn, [-1, 4*4*4*512])
        dense = tf.layers.dense(inputs=flattening, units=1024, activation=tf.nn.relu, trainable=bool(stop_config[11]), name="full_con", reuse=tf.AUTO_REUSE)
        # (1-keep_rate) is the probability that the node will be kept
        # training & trainable are different logic
        dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=bool(stop_config[12]), name="dropout")
        
    with tf.name_scope("y_conv"):
        y_conv = tf.layers.dense(inputs=dropout, units=label_size, name="y_pred", trainable=bool(stop_config[13]), reuse=tf.AUTO_REUSE)
    
    return y_conv

In [315]:
# one hot indexes

import os
model_path = os.path.join(os.getcwd(), 'trained_model', 'model-4')

previous_vars = get_previous_vars(model_path)

learning_rate = 0.001
keep_rate = 0.5

device_name = '/cpu:0' 

n_classes=5

with tf.Session(graph=tf.Graph()) as sess:
    with tf.device(device_name):
        with tf.name_scope('inputs'):
            x_input = tf.placeholder(tf.float32, shape=[None, 32, 32, 32, 1], name="x_input")
            y_input = tf.placeholder(tf.float32, shape=[None, n_classes], name="y_input") 

        prediction = cnn3d_model(x_input, n_classes, seed=1234, keep_rate=0.5)
        tf.add_to_collection("logits", prediction)

        with tf.name_scope("cross_entropy"):
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input), name="cross_entropy")

        with tf.name_scope("training"):
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
            tf.add_to_collection("optimizer", optimizer)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="acc")

    restore_map = {variable.op.name : variable for variable in tf.global_variables() if variable.op.name in previous_vars}

    tf.contrib.framework.init_from_checkpoint(model_path, restore_map)

    sess.run(tf.global_variables_initializer())

    y = sess.run(prediction, feed_dict={x_input: [whole_vox]})
    print(tf.argmax(y, axis=1).eval())
#     probs = []
#     ys = []
#     for _, val in enumerate(vox_segs):
#         y = sess.run(prediction, feed_dict={x_input: [val]})
#         ys.append(y)
#         probs.append(tf.nn.softmax(y).eval())

INFO:tensorflow:Restoring parameters from /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4


INFO:tensorflow:Restoring parameters from /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4


INFO:tensorflow:Initialize variable conv1/bias/Adam_1:0 from checkpoint /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4 with conv1/bias/Adam_1


INFO:tensorflow:Initialize variable conv1/bias/Adam_1:0 from checkpoint /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4 with conv1/bias/Adam_1


INFO:tensorflow:Initialize variable conv2/bias:0 from checkpoint /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4 with conv2/bias


...

INFO:tensorflow:Initialize variable conv5/kernel:0 from checkpoint /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4 with conv5/kernel


INFO:tensorflow:Initialize variable conv5/kernel:0 from checkpoint /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4 with conv5/kernel


INFO:tensorflow:Initialize variable conv7/kernel:0 from checkpoint /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4 with conv7/kernel


INFO:tensorflow:Initialize variable conv7/kernel:0 from checkpoint /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4 with conv7/kernel


[3]
[array([[  0.00000000e+00,   0.00000000e+00,   5.75125817e-18,
          1.00000000e+00,   0.00000000e+00]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          3.04320967e-27,   0.00000000e+00]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  1.,  0.]]

In [310]:
y

array([[-3551.70117188, -1068.67895508,   408.73684692,   531.11138916,
         -842.57104492]], dtype=float32)

In [307]:
count_v(probs)

{2: 13, 3: 305, 4: 4}

In [56]:
def write_to_file(xyz, filename='xyz'):
    import os
    file = open(os.path.join(os.getcwd(), filename + ".pts"), "w") 
    
    for point in xyz:
        st = ""
        for item in point:
            st += str(item) + " "
        file.write(st.strip() + "\n")

    file.close() 
    
write_to_file(segmented_point_cloud)

In [285]:
write_to_file(segs['data'][0])
se_point_cloud = PyntCloud.from_file(os.path.join(os.getcwd(), 'xyz.pts'), sep=" ", header=0, names=["x","y","z"])

In [286]:
se_point_cloud.plot(point_size=0.003)

# produce multi data

In [301]:
my_point_cloud_a = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'airplane', 'points', '1a74b169a76e651ebc0909d98a1ff2b4.pts'), sep=" ", header=0, names=["x","y","z"])
my_point_cloud_b = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'lamp', 'points', '1a9c1cbf1ca9ca24274623f5a5d0bcdc.pts'), sep=" ", header=0, names=["x","y","z"])

print(type(my_point_cloud_a.xyz))
c = np.concatenate((my_point_cloud_a.xyz, my_point_cloud_b.xyz + 0.5))
print(c.shape)

write_to_file(c, filename="ttt")

<class 'numpy.ndarray'>
(5168, 3)


In [302]:
PyntCloud.from_file(os.path.join(os.getcwd(), 'ttt.pts'), sep=" ", header=0, names=["x","y","z"]).plot(point_size=0.01)

# Testing

In [91]:
def find_ranges(point_cloud):

    x_max = x_min = point_cloud[0][0]
    y_max = y_min = point_cloud[0][1]
    z_max = z_min = point_cloud[0][2]
    
    for ind, coor in enumerate(point_cloud):
        if coor[0] > x_max:
            x_max = coor[0]
        if coor[0] < x_min:
            x_min = coor[0]
        if coor[1] > y_max:
            y_max = coor[1]
        if coor[1] < y_min:
            y_min = coor[1]
        if coor[2] > z_max:
            z_max = coor[2]
        if coor[2] < z_min:
            z_min = coor[2]
            
    return ((x_min, x_max), (y_min, y_max), (z_min, z_max))


def find_cube_length(t_range, overall_size):
    length = (t_range[1] - t_range[0])/overall_size
    return length

def get_all_length(point_cloud, overall_size, xyz_ranges):
    
    x_length = find_cube_length(xyz_ranges[0], overall_size[0])
    y_length = find_cube_length(xyz_ranges[1], overall_size[1])
    z_length = find_cube_length(xyz_ranges[2], overall_size[2])
    
    return (x_length, y_length, z_length)
    
def find_target_area_range(point_cloud, target_area, xyz_length, xyz_ranges):
    
    target_x_range_max = xyz_ranges[0][1] - target_area[0][0]*xyz_length[0]
    target_x_range_min = xyz_ranges[0][1] - target_area[0][1]*xyz_length[0]
    target_y_range_max = xyz_ranges[1][1] - target_area[1][0]*xyz_length[1]
    target_y_range_min = xyz_ranges[1][1] - target_area[1][1]*xyz_length[1]
    target_z_range_max = xyz_ranges[2][1] - target_area[2][0]*xyz_length[2]
    target_z_range_min = xyz_ranges[2][1] - target_area[2][1]*xyz_length[2]
    
    return (target_x_range_min, target_x_range_max), (target_y_range_min, target_y_range_max), (target_z_range_min, target_z_range_max)


def segment_points(point_cloud, target_ranges):
    """
    Recives orig point cloud data and a tuple of boundary tuples like:
        ((x_min, x_max), (y_min, y_max), (z_min, z_max))
    """
    
    target_x_range, target_y_range, target_z_range = target_ranges[0], target_ranges[1], target_ranges[2]
    
    points = []
    for ind, coor in enumerate(point_cloud):
        if coor[0] >= target_x_range[0] and coor[0] <= target_x_range[1]:
            if coor[1] >= target_y_range[0] and coor[1] <= target_y_range[1]:
                if coor[2] >= target_z_range[0] and coor[2] <= target_z_range[1]:
                    points.append(coor)
                    
    return np.asarray(points)

In [ ]:
def diagonal_cubes(point_cloud, overall_size=(16,16,16)):
    
    xyz_range = find_ranges(point_cloud)
    xyz_length = get_all_length(point_cloud, overall_size, xyz_range)
    
    